<a href="https://colab.research.google.com/github/christinium/NZ-workshop/blob/master/6%20-%20Obese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 - Obese


In [0]:
#Load libraries that will be needed
import pandas as pd
import numpy as np
import os

from google.colab import auth
from google.cloud import bigquery
from google.colab import files

In [0]:
auth.authenticate_user() #This will allow you to authenticate access to BigQuery

In [0]:
#This is a method that executes a desired SQL query on the database
project_id='new-zealand-2018-datathon'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

In [0]:
# Now load the data. In general you'd load the whole set of notes but that would take
# several minutes so for this example we're just going to use a subset
#notes = pd.read_csv('D.csv')
notes = run_query('''
    SELECT *
    FROM `nlp_demo.obesity_discharge`
    ''')


In [0]:
notes.head()

,row_id,subject_id,chartdate,category,description,text,max
0,11397,169,2131-01-21,Discharge summary,Report,Admission Date: [**2131-1-12**] ...,2131-01-21
1,23645,252,2133-08-19,Discharge summary,Report,Admission Date: [**2133-8-15**] ...,2133-08-19
2,8205,402,2156-11-18,Discharge summary,Report,Admission Date: [**2156-11-11**] ...,2156-11-18
3,47329,433,2164-08-17,Discharge summary,Report,Admission Date: [**2164-8-13**] ...,2164-08-17
4,11128,653,2117-07-15,Discharge summary,Report,Admission Date: [**2117-7-5**] D...,2117-07-15


In [0]:
# Load the gold standard
#gold = pd.read_csv('obese-gold.csv')
gold = run_query('''
    SELECT *  
    FROM `nlp_demo.obese_gold` 
    ''')


In [0]:
# Here is the list of terms we are going to consider "good"
terms = ['obese']

In [0]:
# Now scan through all of the notes. Do any of the terms appear? If so stash the note 
# id for future use

matches = []

for index, row in notes.iterrows():
    if any(x in row['text'] for x in terms):
        matches.append(row['subject_id'])

print("Found " + str(len(matches)) + " matching notes.")

Found 17 matching notes.


In [0]:
# For the patients in those notes, set "obese" true (1) in a the results
myscores = gold.copy()
myscores['obese_1'] = 0 # This sets them all to unknown

for subject_id in matches:
    myscores.loc[myscores["subject_id"] == subject_id,'obese_1'] = 1


In [0]:
# Compute your score

skipped = 0
truepositive = 0
falsepositive = 0
truenegative = 0
falsenegative = 0

for index, row in myscores.iterrows():
    if row['obese_1'] == 0:
        skipped = skipped + 1
    else:
        if row['obese_1'] == 1 and gold.loc[index]['obese_1'] == 1:
            truepositive = truepositive + 1
        elif row['obese_1'] == -1 and gold.loc[index]['obese_1'] == -1:
            truenegative = truenegative + 1
        elif row['obese_1'] == 1 and gold.loc[index]['obese_1'] == -1:
            falsepositive = falsepositive + 1
        elif row['obese_1'] == -1 and gold.loc[index]['obese_1'] == 1:
            falsenegative = falsenegative + 1

print ("Skipped:\t" + str(skipped))
print ("True Pos:\t" + str(truepositive))
print ("True Neg:\t" + str(truenegative))
print ("False Pos:\t" + str(falsepositive))
print ("False Neg:\t" + str(falsenegative))     
print ("SCORE:\t\t" + str(truepositive + truenegative - falsepositive - falsenegative))

Skipped:	83
True Pos:	16
True Neg:	0
False Pos:	1
False Neg:	0
SCORE:		15
